In [3]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Accessing environment variables
user = os.getenv('DB_USER_PROD')
password = os.getenv('DB_PASSWORD_PROD')
host = os.getenv('DB_HOST_PROD')
port = os.getenv('DB_PORT_PROD')  # Default PostgreSQL port is 5432, but we're using a custom one here
db_name = os.getenv('DB_NAME_PROD')

# Create the connection URL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'

# Create the engine
engine_prod = create_engine(connection_string)


In [7]:
df_rev = pd.read_sql_table("qfdmo_revisionacteur",engine_prod)
df_act = pd.read_sql_table("qfdmo_acteur",engine_prod)

/Users/hamzaa/data/quefairedemesobjets/venv/lib/python3.9/site-packages/pandas/io/sql.py:1665: SAWarning: Did not recognize type 'geometry' of column 'location'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)
/Users/hamzaa/data/quefairedemesobjets/venv/lib/python3.9/site-packages/pandas/io/sql.py:1665: SAWarning: Did not recognize type 'geometry' of column 'location'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [16]:
filtered_df = df_act[~df_act['identifiant_unique'].str.startswith("refashion_") & (df_act['source_id'] == 45)]
df_rev_man = pd.merge(filtered_df[['identifiant_unique','identifiant_externe']],df_rev, on = ['identifiant_unique'])
df_rev_man.rename(columns={'identifiant_externe_x': 'identifiant_externe'}, inplace=True)


In [17]:
df_rev_man.to_sql("rev_refashion",engine_prod, if_exists='replace')

77

In [18]:
filtered_df.to_sql("refashion_actors",engine_prod, if_exists='replace')

930

In [ ]:
import psycopg2
from psycopg2 import sql

# Connect to the database again
conn = psycopg2.connect(
    dbname=db_name, 
    user=user, 
    password=password, 
    host=host,
    port=port
)
conn.autocommit = True
cursor = conn.cursor()
delete_queries = [
            """
            DELETE FROM qfdmo_propositionservice_sous_categories
            WHERE propositionservice_id IN (
                SELECT id FROM qfdmo_propositionservice
                WHERE acteur_id IN (
                    SELECT identifiant_unique FROM refashion_actors
                )
            );
            """,
            """
                 DELETE FROM qfdmo_acteur_labels
                  WHERE acteur_id IN (
                         SELECT identifiant_unique FROM refashion_actors
                      );
            """,
            """
            DELETE FROM qfdmo_propositionservice
            WHERE acteur_id IN (
                SELECT identifiant_unique FROM refashion_actors
            );
            """,
            """
            DELETE FROM qfdmo_acteur WHERE identifiant_unique
            in ( select identifiant_unique from refashion_actors);
            """,
            """
            DELETE FROM qfdmo_revisionacteur WHERE identifiant_unique
            in ( select identifiant_unique from refashion_actors);
            """
        ]

for query in delete_queries:
    cursor.execute(query)

cursor.close()
conn.close()

### Validate th dagrun for refashion to trigger but stop create final actors  ---> once it's done proceed below 

In [ ]:
df_act = pd.read_sql_table("qfdmo_acteur",engine_prod)
df_rev = pd.merge(df_act[['identifiant_unique','identifiant_externe']], df_rev_man, on = ['identifiant_externe'])
df_rev.rename(columns={'identifiant_unique_x': 'identifiant_unique'}, inplace=True)
df_rev_man[
            [
                "identifiant_unique",
                "nom",
                "adresse",
                "adresse_complement",
                "code_postal",
                "ville",
                "url",
                "email",
                "location",
                "telephone",
                "nom_commercial",
                "nom_officiel",
                "siret",
                "identifiant_externe",
                "acteur_type_id",
                "statut",
                "cree_le",
                "modifie_le",
                "naf_principal",
                "commentaires",
                "horaires_osm",
                "horaires_description",
                "description",
            ]
        ].to_sql('qfdmo_revisionacteur',engine,            index=False,
if_exists="append",method="multi",
                chunksize=1000)

#### run create final actors